In [1]:
import pandas as pd
import numpy as np

data=pd.read_csv(r'C:\Users\vmffl\workspace\우리은행 데이터경진대회 데이터셋\우리은행 데이터경진대회 데이터셋.csv',index_col=0)

for col_name in data.columns:
    data[col_name]=pd.Categorical(data[col_name])

from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(data,test_size=0.2, random_state=100)

X_trn=train_data.iloc[:,1:]
y_trn=train_data['TARGET']
X_tes=test_data.iloc[:,1:]
y_tes=test_data['TARGET']

### SMOTEN을 이용한 Oversampling

In [2]:
from imblearn.over_sampling import SMOTEN
smoten = SMOTEN(random_state=1000)
X_trn_over, y_trn_over = smoten.fit_resample(X_trn,y_trn)

### 명목형 데이터에 대해서 더미코딩

In [4]:
nominal_col=X_trn_over.columns[[42,43,44,63,64,65,66,67]]
X_trn_over_dummy = pd.get_dummies(X_trn_over,columns=nominal_col)

In [13]:
X_tes_dummy=pd.get_dummies(X_tes,columns=nominal_col)

In [6]:
X_trn_over_dummy.columns

Index(['J_F1_LON_RK_3M_CNT_RE', 'J_F1_OTHER_LON_6M_AVBL_AM_RE',
       'J_F1_OTHER_LON_MAX_3M_AM_RE', 'J_SHINBO_GIBO_AM_RE',
       'J_TOT_LON_CNT_RE', 'J_TOT_LON_AGG_AM_RE', 'J_TOT_LON_MAX_AM_RE',
       'J_DPS_CT_BAL_AM_RE', 'J_TOT_DPS_1M_AVBL_AM_RE',
       'J_TOT_DPS_6M_AVBL_AM_RE', 'J_CORE_DPS_1M_AVBL_AM_RE',
       'J_CORE_DPS_6M_AVBL_AM_RE', 'J_TOT_DPS_ACC_CNT_RE',
       'J_NRML_DPS_ACC_CNT_RE', 'J_TOT_SAVING_ACC_CNT_RE',
       'J_NRML_SAVING_ACC_CNT_RE', 'J_TOT_TS_1M_CNT_RE', 'J_TOT_TS_6M_CNT_RE',
       'J_TOT_TS_1M_AM_RE', 'J_TOT_TS_6M_AM_RE', 'J_OB_TS_1M_CNT_RE',
       'J_OB_TS_6M_CNT_RE', 'J_OB_TS_1M_AM_RE', 'J_OB_TS_6M_AM_RE',
       'J_IB_TS_1M_CNT_RE', 'J_IB_TS_6M_CNT_RE', 'J_IB_TS_1M_AM_RE',
       'J_IB_TS_6M_AM_RE', 'J_TOT_TS_3M_AVG_CNT_RE', 'J_TOT_TS_6M_AVG_CNT_RE',
       'J_TOT_TS_3M_AVG_AM_RE', 'J_TOT_TS_6M_AVG_AM_RE',
       'J_OB_TS_3M_AVG_CNT_RE', 'J_OB_TS_6M_AVG_CNT_RE',
       'J_OB_TS_3M_AVG_AM_RE', 'J_OB_TS_6M_AVG_AM_RE', 'J_IB_TS_3M_AVG_CNT_RE',
       

In [8]:
X_trn_over.shape[1]

92

In [9]:
X_trn_over_dummy.shape[1]

100

In [10]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score
from sklearn.metrics import f1_score, confusion_matrix, precision_recall_curve, roc_curve

def get_eval(y_test,pred=None):
    confusion=confusion_matrix(y_test,pred)
    accuracy=accuracy_score(y_test,pred)
    precision=precision_score(y_test,pred)
    recall=recall_score(y_test, pred)
    f1 = f1_score(y_test,pred)
    precision2=precision_score(y_test,pred,pos_label=0)
    recall2=recall_score(y_test,pred,pos_label=0)
    f1_2=f1_score(y_test,pred,pos_label=0)
    print("confusion matrix")
    print(confusion)
    print('accuracy : {0:.4f} , precision : {1:.4f} , recall : {2:.4f}, F1 score : {3:.4f} \n precision(0) : {4:.4f} , recall(0) : {5:.4f}, f1(0) : {6:.4f}'.format(accuracy,precision, recall,f1,precision2,recall2,f1_2))

### logistic regression 성능비교

In [11]:
from sklearn.linear_model import LogisticRegression
clf=LogisticRegression(max_iter = 1000)
clf.fit(X_trn_over,y_trn_over)
pred=clf.predict(X_tes)
get_eval(y_tes,pred)

confusion matrix
[[20862  2889]
 [   35   121]]
accuracy : 0.8777 , precision : 0.0402 , recall : 0.7756, F1 score : 0.0764 
 precision(0) : 0.9983 , recall(0) : 0.8784, f1(0) : 0.9345


In [14]:
clf2=LogisticRegression(max_iter=1000)
clf2.fit(X_trn_over_dummy,y_trn_over)
pred2=clf2.predict(X_tes_dummy)
get_eval(y_tes,pred2)

confusion matrix
[[20862  2889]
 [   35   121]]
accuracy : 0.8777 , precision : 0.0402 , recall : 0.7756, F1 score : 0.0764 
 precision(0) : 0.9983 , recall(0) : 0.8784, f1(0) : 0.9345


### Naive bayes 성능비교

In [17]:
from sklearn.naive_bayes import CategoricalNB
catNB=CategoricalNB(fit_prior=False)
catNB.fit(X_trn_over,y_trn_over)
pred3=catNB.predict(X_tes)
get_eval(y_tes,pred3)

confusion matrix
[[19408  4343]
 [   59    97]]
accuracy : 0.8159 , precision : 0.0218 , recall : 0.6218, F1 score : 0.0422 
 precision(0) : 0.9970 , recall(0) : 0.8171, f1(0) : 0.8981


In [18]:
catNB2=CategoricalNB(fit_prior=False)
catNB2.fit(X_trn_over_dummy,y_trn_over)
pred4=catNB2.predict(X_tes_dummy)
get_eval(y_tes,pred4)

confusion matrix
[[19337  4414]
 [   57    99]]
accuracy : 0.8130 , precision : 0.0219 , recall : 0.6346, F1 score : 0.0424 
 precision(0) : 0.9971 , recall(0) : 0.8142, f1(0) : 0.8964


### Random Forest 성능비교

In [19]:
from sklearn.ensemble import RandomForestClassifier

rdf_clf = RandomForestClassifier(random_state=1000)
rdf_clf.fit(X_trn_over, y_trn_over)
pred5 = rdf_clf.predict(X_tes)
get_eval(y_tes,pred5)

confusion matrix
[[23739    12]
 [   80    76]]
accuracy : 0.9962 , precision : 0.8636 , recall : 0.4872, F1 score : 0.6230 
 precision(0) : 0.9966 , recall(0) : 0.9995, f1(0) : 0.9981


In [20]:
rdf_clf2=RandomForestClassifier(random_state=1000)
rdf_clf2.fit(X_trn_over_dummy,y_trn_over)
pred6 = rdf_clf2.predict(X_tes_dummy)
get_eval(y_tes,pred6)

confusion matrix
[[23738    13]
 [   81    75]]
accuracy : 0.9961 , precision : 0.8523 , recall : 0.4808, F1 score : 0.6148 
 precision(0) : 0.9966 , recall(0) : 0.9995, f1(0) : 0.9980
